<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>

<br>

<a href="./23_ltpy_Sentinel5p_L2_data.ipynb"><< 23 - Copernicus Sentinel-5 Precursor (Sentinel-5P) </a><span style="float:right;"><a href="./25_ltpy_Sentinel3_OLCI_L1.ipynb">25 - Sentinel-3 OLCI Level 1 data>></a></span>

# 2.4 Copernicus Atmosphere Monitoring Service (CAMS) data

The Copernicus Atmopshere Monitoring Service (CAMS) provides consistent and quality-controlled information related to air pollution and health and greenhouse gases. CAMS data consist of `global forecasts and analyses`, `global reanalyses`, `fire emissions` and `greenhouse gas flux inversions`.

#### Module outline:
* [1 - Load, browse and plot CAMS fire emissions data](#cams_gfas)
* [2 - Example - CAMS regional forecast data](#cams_regional)
* [3 - Load, browse and plot CAMS greenhouse gas flux inversions](#load_ggf)

#### Load required libraries

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.cm import get_cmap
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [ ]:
from ipynb.fs.full.ltpy_functions import visualize_s5p_pcolormesh

<hr>

## <a id="cams_gfas"></a>Load, browse and plot CAMS fire emissions data

### Open a CAMS GFAS netCDF file with `xarray`

CAMS GFAS fire emission data can be retrieved in either `GRIB` or `NetCDF` format. With the Python library `xarray` and the `open_dataset` function, we can easily read a single `NetCDF` file.

CAMS GFAS fire emission data are three dimensional data, with the dimensions `latitude`, `longitude` and `time`. The data file loaded has three time steps, from 22 September to 24 September 2019 and a global spatial coverage. The xarray dataset contains a data variable called `frpfire`.

In [ ]:
gfas_frpfire_xr = xr.open_dataset('./eodata/cams/gfas/2019/10/20191022-24_gfas_radiative_power.nc')
gfas_frpfire_xr

We can select the data variable with `[]`, which gives us acces to the DataArray and more parameter attributes. Thus, the dataset values are `wildfire radiative power` and the parameter unit is `W m**-2`.

In [ ]:
frpfire = gfas_frpfire_xr['frpfire']
frpfire

With xarray's `where` function, we can build a geographical subset. Let's define a bounding box for south-east asia `[30,160,-20,80]` and subset the data. We see that the data has a negative _FillValue. Thus a next step is to filter negative values and set them to NaN.

In [ ]:
latmin=-20
latmax=30
lonmin=80
lonmax=160
frpfire_subset = frpfire.where((frpfire.latitude < latmax) & (frpfire.latitude > latmin) & (frpfire.longitude < lonmax) & (frpfire.longitude > lonmin),drop=True)
frpfire_subset

In [ ]:
frpfire_subset.values[frpfire_subset.values<0]=np.nan
frpfire_subset

### Visualize CAMS GFAS fire emissions data

You can use the function [visualize_s5p_pcolormesh](./ltpy_functions.ipynb#visualize_s5p_pcolormesh) again. This time, you should set a logarithmic color scaling with setting `log_scale=True`. 
With `matplotlib.colors.ListedColorMap`, you can specify an individual color scale. Let's call the color scale `cmap` and plot the fire emissions data. Alternatively, you can use the predefine color scale `hot`.

In [ ]:
cmap = matplotlib.colors.ListedColormap(['#330305','#620103','#880105','#B00602','#DA0302','#FF0907','#FD3304','#FF3106', 
                                  '#FF5A02', '#FC8706','#FDB004','#FADD02','#FFFF0C','#FEFF47','#FFFE85','#FFFEC5',
                                  '#FFFFFF'])

In [ ]:
unit = frpfire.units
longname= frpfire.long_name

In [ ]:
visualize_s5p_pcolormesh(frpfire_subset.isel(time=0).data, frpfire_subset.longitude.data, frpfire_subset.latitude.data, ccrs.PlateCarree(), 'hot', unit, longname + ' ' + str(frpfire_subset.isel(time=0).time.data), 0.0001, 5, lonmin, lonmax, latmin, latmax, log=True, set_global=False)
  

<br>

## <a id='cams_regional'></a>Example - CAMS regional forecast data

### Load CAMS station location data

CAMS data offers regional forecast and station location data in `csv`. With the help of the Python library [Pandas](https://pandas.pydata.org/) and its function `read_csv`, csv files can easily be read.
You can read the station locations and with `len()` you can see that it contains 2949 station data.

In [ ]:
station_locations = 'eodata/cams/regional/csv/CAMS_WEB_LOCATIONS_V1.csv'

locations = pd.read_csv(station_locations)
len(locations)

Pandas' `head()` function allows you to get the first couple of entries of the list. This is helpful to get an overview of the csv content. You see that the csv contains station `ID`, `name`, `Country`, `Latitude` and `Longitude` information.

In [ ]:
locations.head()

You can select a city and filter for entries for this location. Let's make an example with `Paris`. You can see that the dataset contains two entries, where the station `Name` contains the string `Paris`.

In [ ]:
city = 'Paris'
locations[locations['Name'].str.contains(city)]

### Load CAMS regional forecast data

The same principle you can apply to load and read the regional forecast data. Let's use Pandas' `read_csv` function. By specifying a `DATE`, you can parse the file that contains the data for this specific day. Let's do an example to filter the data for `10 May 2018`.

You see that the `csv` contains measurements of `O3`, `NO2`, `SO2`, `CO`, `PM10` and `PM2.5`.

In [ ]:
DATE='20180510'
DPLUS=1

datafile = 'eodata/cams/regional/csv/CAMS_WEB_FORECAST_%s_D+%d.csv.gz'
path = datafile % (DATE, DPLUS)
aq_data = pd.read_csv(path, index_col=['ID', 'Date_Time'], parse_dates=[['Date','Time']] )
aq_data.head()

### Visualize regional `Air Quality` forecast data for one specific station location

We can now filter the station ID for `Paris-Charles-de-Gaulle` and visualize all `Air Quality` forecast parameters for `11 May 2018`.

In [ ]:
aq_data.loc['FRXX0077'].plot.line(figsize=(15,8),linestyle='dashed')

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.title('CAMS regional forecast data - 11 May 2018', fontsize=20, pad=20)

plt.ylabel('Unit', fontsize=16)
plt.xlabel('Hour', fontsize=16)

plt.legend(fontsize=16,loc=1)
plt.show()

<br>

## <a id="load_ggf"></a>Load, browse and plot CAMS Greenhouse Gas Fluxes

### Load the greenhouse gas flux inversion dataset with `netCDF.Dataset`

With Python's library `netCDF` and the `Dataset` constructor, you can easily load `NetCDF` data. Let's load the data for January 1979. You can see that the dataset has two dimensions, `latitude` and `longitude` and a range of different 2-dimensional `variables`.

In [ ]:
year = 1979
month_ = 1
month = "{}{:02d}".format(year, month_)
dataset = nc.Dataset("./eodata/cams/cams_ghg_fluxes/z_cams_l_lsce_{}_v18r2_ra_sfc_mm_co2flux.nc".format(month))
dataset

With `[]`, we can select a variable we are interested in. Let's load for example `flux_apos_bio` and `latitude` and `longitude` information to get more information about the extent.

You can see that the `flux_apos_bio` is the `Posterior land surface upward mass flux of carbon for the whole grid box and the whole month without fossile`. The variable unit is `kgC / m2 month`.

The dataset has a global extent.

In [ ]:
co2_flux = dataset.variables['flux_apos_bio']
lats = dataset.variables['latitude']
lons = dataset.variables['longitude']

co2_flux, lats[:], lons[:]

### Visualize a ghg flux inversion variable with `pcolormesh`

Now, we can plot the `Carbon upward flux` for a geographical subset, e.g. a region in Africa.

In [ ]:
fig,ax = plt.subplots(figsize=(20,12))
ax = plt.axes(projection=ccrs.PlateCarree())

extent = [5, 35, -15, 10]
ax.set_extent(extent)

ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
ax.add_feature(cartopy.feature.RIVERS)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-')

gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.xlabels_top=False
gl.ylabels_right=False
gl.xformatter=LONGITUDE_FORMATTER
gl.yformatter=LATITUDE_FORMATTER

gl.xlabel_style={'size':14}
gl.ylabel_style={'size':14}

cmap = get_cmap("PiYG_r", 21)
vmin = -.12
vmax = .12
clevs = np.arange(vmin, vmax, 0.01)
img = plt.pcolormesh(lons, lats, co2_flux[:,:], cmap=cmap, vmin=vmin, vmax=vmax)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

cbar = fig.colorbar(img, ax=ax, orientation='vertical', fraction=0.04, pad=0.05)
cbar.set_label(co2_flux.units, fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.title(co2_flux.long_name.format(month), fontsize=20, pad=20.0)

plt.show()

<br>

<br>

<a href="./23_ltpy_Sentinel5p_L2_data.ipynb"><< 23 - Copernicus Sentinel-5 Precursor (Sentinel-5P) </a><span style="float:right;"><a href="./25_ltpy_Sentinel3_OLCI_L1.ipynb">25 - Sentinel-3 OLCI Level 1 data>></a></span>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/atmosphere">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>